In [5]:
## Imports and get_param

import os
import sys

import h5py
import torch
from torch import optim
import numpy as np
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join('..')))

from src.models.AngularCentralGauss_torch import AngularCentralGaussian as ACG
from src.models.Watson_torch import Watson
from src.models.MixtureModel_torch import TorchMixtureModel
from src.models.HMM_torch import HiddenMarkovModel as HMM
from src.various.training import train_hmm

def get_param(model, show=True):
    para = model.get_model_param()
    
    if show:
        for p_k in para:
            print(p_k)
            print(para[p_k])
            print(10*'---')
    
    return para

## Load data and get correct shape and dtype

In [6]:
synth_dataset = '../data/synthetic/HMMdata.h5'
dataf = h5py.File(synth_dataset, mode='r')
data = torch.tensor(np.array(dataf['X']))
data = torch.unsqueeze(torch.transpose(data,dim0=0,dim1=1),dim=0).float()
print(data.shape) #needs to be subjects, time, dims
print(data.dtype)
print(torch.norm(data,dim=2))

torch.Size([1, 2880, 3])
torch.float32
tensor([[1., 1., 1.,  ..., 1., 1., 1.]])


## ACG mixture
train model with diff learning rates, get best model 

In [7]:
best_LR = 0.1
int_epoch = 1

ACG_MM = TorchMixtureModel(distribution_object=ACG,K=2, dist_dim=3)
ACG_HMM = HMM(num_states=2, observation_dim=3, emission_dist=ACG)
Watson_MM = TorchMixtureModel(distribution_object=Watson,K=2, dist_dim=3)
Watson_HMM = HMM(num_states=2, observation_dim=3, emission_dist=Watson)

ACG_MM_optimizer = optim.Adam(ACG_MM.parameters(), lr=best_LR)
ACG_MM_ll = train_hmm(ACG_MM, data=torch.squeeze(data), optimizer=ACG_MM_optimizer, num_epoch=int_epoch, keep_bar=False)

ACG_HMM_optimizer = optim.Adam(ACG_HMM.parameters(), lr=best_LR)
ACG_HMM_ll = train_hmm(ACG_HMM, data=data, optimizer=ACG_HMM_optimizer, num_epoch=int_epoch, keep_bar=False)

Watson_MM_optimizer = optim.Adam(Watson_MM.parameters(), lr=best_LR)
Watson_MM_ll = train_hmm(Watson_MM, data=torch.squeeze(data), optimizer=Watson_MM_optimizer, num_epoch=int_epoch, keep_bar=False)

Watson_HMM_optimizer = optim.Adam(Watson_HMM.parameters(), lr=best_LR)
Watson_HMM_ll = train_hmm(Watson_HMM, data=data, optimizer=Watson_HMM_optimizer, num_epoch=int_epoch, keep_bar=False)


RuntimeError: bad allocation

## get best models

In [11]:
#acg_mm_idx = np.argmax(acg_mm_LR_results)
#acg_hmm_idx = np.argmax(acg_hmm_LR_results)
#watson_mm_idx = np.argmax(watson_mm_LR_results)
#watson_hmm_idx = np.argmax(watson_hmm_LR_results)

0


## get emission probs and viterbi

In [ ]:
#best_paths, paths_probs, emission_probs = best_model.viterbi2(data)
#np.savetxt('../data/synthetic/emissionprobs_ACG.csv', emission_probs, delimiter=',')
#np.savetxt('../data/synthetic/best_path_ACG.csv', best_paths, delimiter=',')

## extract parameters from the best model (should be pi=0.5)

In [12]:
#acgbest_param = get_param(best_model)
#learned_sigma = torch.stack([acgbest_param[f'emission_model_{idx}'] for idx in range(best_model.N)])
#learned_pi = acgbest_param['un_norm_priors']
#learned_pi = torch.nn.functional.softmax(learned_pi,dim=0)
#print(learned_pi)

un_norm_pi
tensor([ 0.9407, -1.0398])
------------------------------
mix_comp_0
{'mu': tensor([0.0953, 0.6978, 0.7099]), 'kappa': tensor([5.0048])}
------------------------------
mix_comp_1
{'mu': tensor([0.5667, 0.5848, 0.5804]), 'kappa': tensor([9.9425])}
------------------------------
